In [36]:
import os
from collections import Counter
from typing import List, Tuple
import ujson as json  # Usar ujson para un análisis JSON más rápido
import emoji
from concurrent.futures import ThreadPoolExecutor

# Función para procesar un grupo de líneas y contar emojis
def process_lines(lines: List[str]) -> Counter:
    emoji_counter = Counter()
    for line in lines:
        try:
            data = json.loads(line)
            content = data.get("content", "")
            if content:
                for value in emoji.analyze(content):
                    emoji_counter[value.chars] += 1
        except (json.JSONDecodeError, KeyError):
            continue
    return emoji_counter

def q2_time(file_path: str) -> List[Tuple[str, int]]:
    # Leer el archivo línea por línea
    with open(file_path, 'r') as file_data:
        lines = file_data.readlines()
    
    # Dividir las líneas en grupos para procesamiento en paralelo
    num_threads = 24  # Ajustar basado en el número de núcleos de CPU
    total_lines = len(lines)
    chunk_size = (total_lines // num_threads) + (total_lines % num_threads > 0)  # Redondear hacia arriba
    
    # Usar ThreadPoolExecutor para paralelizar el procesamiento de líneas
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = [executor.submit(process_lines, lines[i:i + chunk_size]) for i in range(0, total_lines, chunk_size)]
    
    # Reunir los resultados de todos los hilos
    total_counter = Counter()
    for future in futures:
        total_counter.update(future.result())
    
    # Clasificar los 10 emojis más usados
    top_10_emojis = total_counter.most_common(10)
    
    return top_10_emojis

# Short file
#file_path='/Users/juanignaciomagarinoscastro/Downloads/farmers-protest-tweets-2021-2-4.json'
# Long file
# Ruta del archivo JSON a procesar
file_path = '/Users/juanignaciomagarinoscastro/Downloads/farmers-protest-tweets-2021-2-4-large.json'
q2_time(file_path)


[('🙏', 50490),
 ('😂', 30720),
 ('🚜', 29720),
 ('🌾', 21820),
 ('🇮🇳', 20860),
 ('🤣', 16680),
 ('✊', 16510),
 ('❤️', 13820),
 ('🙏🏻', 13170),
 ('💚', 10400)]

In [34]:
import psutil
import os
from collections import Counter
from typing import List, Tuple
import ujson as json  # Use ujson for faster JSON parsing
import emoji
from concurrent.futures import ThreadPoolExecutor

# Función para obtener el uso actual de memoria en MB
def memory_usage():
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss / 1024 / 1024  # Convertir a MB

# Función para procesar un grupo de líneas y contar emojis
def process_lines(lines: List[str]) -> Counter:
    emoji_counter = Counter()
    for line in lines:
        try:
            data = json.loads(line)
            content = data.get("content", "")
            if content:
                for value in emoji.analyze(content):
                    emoji_counter[value.chars] += 1
        except (json.JSONDecodeError, KeyError):
            continue
    return emoji_counter

def q2_time(file_path: str) -> List[Tuple[str, int]]:
    print(f"Uso de memoria antes de cargar los datos: {memory_usage()} MB")

    # Leer el archivo línea por línea
    with open(file_path, 'r') as file_data:
        lines = file_data.readlines()
    
    print(f"Uso de memoria después de leer el archivo: {memory_usage()} MB")
    
    # Dividir las líneas en grupos para procesamiento en paralelo
    num_threads = 24  # Ajustar basado en el número de núcleos de CPU
    total_lines = len(lines)
    chunk_size = (total_lines // num_threads) + (total_lines % num_threads > 0)  # Redondear hacia arriba
    
    # Usar ThreadPoolExecutor para paralelizar el procesamiento de líneas
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = [executor.submit(process_lines, lines[i:i + chunk_size]) for i in range(0, total_lines, chunk_size)]
    
    # Reunir los resultados de todos los hilos
    total_counter = Counter()
    for future in futures:
        total_counter.update(future.result())
    
    print(f"Uso de memoria después de procesar los datos: {memory_usage()} MB")
    
    # Clasificar los 10 emojis más usados
    top_10_emojis = total_counter.most_common(10)
    
    print(f"Uso de memoria después de contar los emojis: {memory_usage()} MB")
    
    return top_10_emojis

# Short file
#file_path='/Users/juanignaciomagarinoscastro/Downloads/farmers-protest-tweets-2021-2-4.json'
# Long file
# Ruta del archivo JSON a procesar
file_path = '/Users/juanignaciomagarinoscastro/Downloads/farmers-protest-tweets-2021-2-4-large.json'
q2_time(file_path)


Uso de memoria antes de cargar los datos: 490.734375 MB
Uso de memoria después de leer el archivo: 787.25 MB
Uso de memoria después de procesar los datos: 4406.609375 MB
Uso de memoria después de contar los emojis: 4406.6875 MB


[('🙏', 50490),
 ('😂', 30720),
 ('🚜', 29720),
 ('🌾', 21820),
 ('🇮🇳', 20860),
 ('🤣', 16680),
 ('✊', 16510),
 ('❤️', 13820),
 ('🙏🏻', 13170),
 ('💚', 10400)]